In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from janelia_core.ml.latent_regression import LatentRegModel
from janelia_core.ml.latent_regression import LinearMap

## Parameters for model go here

In [3]:
d_in = [5, 6, 7]
d_out = [6, 9, 10]

d_proj = [1, 2, 4]
d_trans = [1, 3, 3 ]

n_smps = 20000

In [4]:
## Create the model here
n_groups = len(d_in)

M = LinearMap(d_proj, d_trans)
S = [torch.nn.ReLU() for g in range(n_groups)]

mdl = LatentRegModel(d_in, d_out, d_proj, d_trans, M, S, direct_pairs=[(1, 0)])

## Generate some x data here

In [5]:
x = [torch.randn([n_smps, d]) for d in d_in]

## Run the model forward 

In [6]:
y_pred = mdl(x)
y = mdl.generate(x)

## Fit a new model

In [7]:
M_fitted = LinearMap(d_proj, d_trans)
S_fitted = [torch.nn.ReLU() for g in range(n_groups)]
fitted_mdl = LatentRegModel(d_in, d_out, d_proj, d_trans, M_fitted, S_fitted, direct_pairs=[(1, 0)])

In [8]:
log = fitted_mdl.fit(x, y, max_its=1000, update_int=100, min_var = .01)

0: Elapsed fitting time 2.3603439331054688e-05, vl: 94.45749, lr: 0.01
100: Elapsed fitting time 0.18752694129943848, vl: -6.70902, lr: 0.01
200: Elapsed fitting time 0.37792086601257324, vl: -12.492106, lr: 0.01
300: Elapsed fitting time 0.551459789276123, vl: -16.018393, lr: 0.01
400: Elapsed fitting time 0.7409317493438721, vl: -17.129686, lr: 0.01
500: Elapsed fitting time 0.9331817626953125, vl: -17.231886, lr: 0.01
600: Elapsed fitting time 1.1183388233184814, vl: -17.76941, lr: 0.01
700: Elapsed fitting time 1.3127999305725098, vl: -16.972315, lr: 0.01
800: Elapsed fitting time 1.5022227764129639, vl: -16.79217, lr: 0.01
900: Elapsed fitting time 1.6823148727416992, vl: -17.51792, lr: 0.01
999: Elapsed fitting time 1.8704450130462646, vl: -17.344519


In [17]:
%matplotlib qt
plt.figure()
plt.plot(log['elapsed_time'], log['obj'])

In [20]:
true_y_hat = mdl(x)
fitted_y_hat = fitted_mdl(x)

plt_grp = 2
plt_neuron = 0
plt_smps = np.arange(1000)

plt.figure()

true_plt_values = true_y_hat[plt_grp][plt_smps, plt_neuron].detach().numpy()
fitted_plt_values = fitted_y_hat[plt_grp][plt_smps, plt_neuron].detach().numpy()

plt.plot(true_plt_values, 'ro')
plt.plot(fitted_plt_values, 'b-')